Accessing Satellite data:
We will use Sentinel 2 data. There are many options to access Sentinel 2 images and most of them will require you to access through website interaction whether directly via a downloading service utility or via the cloud. However, since we are using Jupyter notebook, we will access them right here using, sentinelsat a python library which makes searching, retrieving and downloading Sentinel satellite images easy. So let us start installing sentinelsat through pip. We also install other packages that we will use as we continue.

Installation of the required Packages

In [5]:
pip install --upgrade pip #you may want to run this so as to avoid getting errors while installing the packages


     ---------------------------------------- 2.0/2.0 MB 45.1 kB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2


In [7]:
!pip install sentinelsat
!pip install folium
!pip install descartes
!pip install rasterio
!pip install geopandas
!pip install pandas


     --------------------------------------- 48.7/48.7 kB 33.8 kB/s eta 0:00:00


Importing Libraries:

In [8]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import pandas as pd
import geopandas as gpd

from os import path as op
import ee
import collections
collections.Callable = collections.abc.Callable
import folium
import numpy as np
import rasterio
from shapely.geometry import MultiPolygon, Polygon
import fiona

import math
import matplotlib # base python plotting library
import matplotlib.pyplot as plt # submodule of matplotlib

# To display plots, maps, charts etc in the notebook
%matplotlib inline

Before we are able to use sentinelsat, we need to register a username in Copernicus Open Access Hub and note down your username and password and paste them here inside the code.

In [9]:
from sentinelsat import SentinelAPI

user = 'lawrence65' #I am sharinmg with you my details for the account so that you may avoid the hussle: you can register later on
password = 'lawrence65' #user credentials
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

You are now set to use sentinelsat and download Sentinel Satellite images. We then use boundary data for Central Guyana as you had shared. We are going to read it with Geopandas.

In [10]:
# Read in the Central_Guyana shapefile
Central_Guyana = gpd.read_file(r'C:\Users\Lawrence\AoI.shp')

One last step before we can search and download sentinel 2 images is to create a footprint from the Central_Guyana Shapefile Geometry. Here we will use Shapely Python library since our data is in Shapefiles and have read it already as Geopandas GeodataFrame. (Note that if you have Geojson data, sentinelsatprovides a handy way to convert your data into a proper format in the query).

In [11]:
from shapely.geometry import MultiPolygon, Polygon

footprint = None
for i in Central_Guyana['geometry']:
    footprint = i

Now we can run a query on the apiwe have created above. There are different ways you can construct your query here depending on your use case. In this example, we will create a query for Sentinel 2 images Level 2A with cloud coverage between 0 and 60 that fall or intersect with the footprint (Area of study:"in our case Central Guyana").NB: You can adjust the percentage depending on your case/Study area. For the time period, we are interested only in Sentinel Level 2A satellite images taken between '20190101' and '20190110’ (For reference on valid search queries please refer to scihub).

In [12]:
products = api.query(footprint,
                     date = ('20190101', '20190110'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0,60)
                    )

Sorting the Sentinel 2 data

In [13]:
products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['cloudcoverpercentage'], ascending=[True])
products_gdf_sorted

,title,link,link_alternative,link_icon,summary,ondemand,ingestiondate,beginposition,endposition,orbitnumber,...,platformidentifier,orbitdirection,platformserialidentifier,processingbaseline,processinglevel,producttype,platformname,size,uuid,geometry
b2fe4dfb-6f87-44ce-b014-ef2f4bce1382,S2A_MSIL2A_20190109T143751_N0211_R096_T20NQN_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-01-09T14:37:51.024Z, Instrument: MS...",false,2019-01-10 01:24:33.690,2019-01-09 14:37:51.024,2019-01-09 14:37:51.024,18540,...,2015-028A,DESCENDING,Sentinel-2A,02.11,Level-2A,S2MSI2A,Sentinel-2,967.02 MB,b2fe4dfb-6f87-44ce-b014-ef2f4bce1382,"POLYGON ((-60.21833 7.22931, -60.23113 7.17295..."
5ca274c8-8c14-4133-ac79-c39bbe838c6f,S2B_MSIL2A_20190104T143749_N0211_R096_T20NPN_2...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2019-01-04T14:37:49.024Z, Instrument: MS...",false,2019-01-04 22:08:11.497,2019-01-04 14:37:49.024,2019-01-04 14:37:49.024,9560,...,2017-013A,DESCENDING,Sentinel-2B,02.11,Level-2A,S2MSI2A,Sentinel-2,1.03 GB,5ca274c8-8c14-4133-ac79-c39bbe838c6f,"POLYGON ((-62.09421 7.23693, -61.09988 7.23386..."


Satellte Imagery Download: (Sentinel 2)

From the results that are acquired above, you can now download your Sentinel 2 data (specific) from the list of outputs using it's ID(depending on which you've settled for) .

In [14]:
api.download("b2fe4dfb-6f87-44ce-b014-ef2f4bce1382")

MD5 checksumming:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

{'id': 'b2fe4dfb-6f87-44ce-b014-ef2f4bce1382',
 'title': 'S2A_MSIL2A_20190109T143751_N0211_R096_T20NQN_20190109T170017',
 'size': 1014050163,
 'md5': '97cc763b6391b34cc0d065777425b7b0',
 'date': datetime.datetime(2019, 1, 9, 14, 37, 51, 24000),
 'footprint': 'POLYGON((-60.21833284844435 7.229308889858343,-60.23112984057468 7.172951534258405,-60.26482735539315 7.024209748964548,-60.29837698427468 6.875500997171449,-60.331919646073935 6.726900280255413,-60.36531511118846 6.57834896049319,-60.398530785590815 6.42980670596945,-60.432759020974046 6.28165965236307,-60.44250661148195 6.238186112249442,-61.192661063507735 6.24146759896048,-61.18898069721401 7.234226743659733,-60.21833284844435 7.229308889858343))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('b2fe4dfb-6f87-44ce-b014-ef2f4bce1382')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2019, 1, 10, 1, 28, 32, 920000),
 'Ingestion Date': datetime.datetime(2019, 1, 10, 1, 24, 33, 690000),
 'quicklook_url':

Exploring the Satellite Imagery:
Time to use python’s Rasterio library since satellite images are grids of pixel-values and can be interpreted as multidimensional arrays.

Reading the Imagery from our local machine:

In [27]:
image_file = "S2A_MSIL2A_20190109T143751_N0211_R096_T20NQN_20190109T170017.zip"  # just add the image downloaded from Sentinelsat.
sat_data = rasterio.open(image_file)

Image dimension in meters as well rows and columns:

In [28]:
width_in_projected_units = sat_data.bounds.right - sat_data.bounds.left
height_in_projected_units = sat_data.bounds.top - sat_data.bounds.bottom
print("Width: {}, Height: {}".format(width_in_projected_units, height_in_projected_units))
print("Rows: {}, Columns: {}".format(sat_data.height, sat_data.width))

Width: 512.0, Height: -512.0
Rows: 512, Columns: 512


Pixel conversion to latitude and longitude:

In [29]:
# Upper left pixel
row_min = 0
col_min = 0
# Lower right pixel.  Rows and columns are zero indexing.
row_max = sat_data.height - 1
col_max = sat_data.width - 1
# Transform coordinates with the dataset's affine transformation.
topleft = sat_data.transform * (row_min, col_min)
botright = sat_data.transform * (row_max, col_max)
print("Top left corner coordinates: {}".format(topleft))
print("Bottom right corner coordinates: {}".format(botright))

Top left corner coordinates: (0.0, 0.0)
Bottom right corner coordinates: (511.0, 511.0)


Bands:
Storing the bands(B,G,R,N infrared) in numpy array:


In [32]:
print(sat_data.count)
# sequence of band indexes
print(sat_data.indexes)

0
()


Raster Mosaic with Python
Let us first import the libraries and create the output folder with Python path lib.For this step you will use satellite imagery bands already downloaded in your local machine e.g Landsat imagery 


In [ ]:
from rasterio.plot import show
from rasterio.merge import merge
import rasterio as rio
from pathlib import Pathpath = Path('data/')
Path('output').mkdir(parents=True, exist_ok=True)
output_path = 'output/mosaic_output.tif'

Now we iterate over the available .tif files in the data folder. We will merge all files in this data folder and create a mosaic from them. We also create an empty list to hold the files in the data folder.

In [ ]:
raster_files = list(path.iterdir())
raster_to_mosiac = []

We then loop through the raster files, open them with rasterio and append them to the raster_to_mosiac list we created above.

In [ ]:
for p in raster_files:
    raster = rio.open(p)
    raster_to_mosiac.append(raster)

From this stage on, it is easy. We use the merge() method from rasterio to create the mosaic. We also create the output transformation parameters to use later.

In [ ]:
mosaic, output = merge(raster_to_mosiac)

Now, we copy the raster's metadata and update it to match the height and width of the mosaic.

In [ ]:
output_meta = raster.meta.copy()
output_meta.update(
    {"driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": output,
    }
)

In this final stage, we write the mosaiced file in a local folder.

In [ ]:
with rio.open(output_path, “w”, **output_meta) as m:
    m.write(mosaic)

And there you have your mosaiced raster image!

Visualizing the Satellite Imagery: Colour Balance:

In [ ]:
# Load the 4 bands into 2d arrays - recall that we previously learned PlanetScope band order is BGRN.
b, g, r, n = sat_data.read()
# Displaying the blue band.
fig = plt.imshow(b)
plt.show()

In [ ]:
# Displaying the green band.
fig = plt.imshow(g)
fig.set_cmap('gist_earth')
plt.show()

In [ ]:
# Displaying the red band.
fig = plt.imshow(r)
fig.set_cmap('inferno')
plt.colorbar()
plt.show()

In [ ]:
# Displaying the infrared band.
fig = plt.imshow(n)
fig.set_cmap('winter')
plt.colorbar()
plt.show()